<a href="https://colab.research.google.com/github/juliosdutra/FundamentosComputacionais/blob/main/1o_dia_ex_deepxde_curso_ermac_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução às Redes Neurais Fisicamente Informadas - PINNs (Dia 01) Picture1.jpg

---
**EXEMPLO 1**

**Objetivo:** resolver o problema de valor inicial dado pela equação diferencial ordinária de 1a ordem $$y'(t)+3t^2 y(t)=6t^2$$ com condição inicial $y(0)=0$.

Esse PVI tem solução dada por $y(t)=2-2e^{-t^3}$ que pode ser obtida pelo método do fator integrante.


---
Começamos instalando a biblioteca DeepXDE, pois ela não é parte do Google Colab.

In [1]:
!pip install deepxde -q gwpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 45.0.5 which is incompatible.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 45.0.5 which is incompatible.


---
E em seguida importando o necessário, perceba que estamos agora importando o DeepXDE como `dde`

O DeepXDE aceita várias APIs de aprendizado de máquina como backend, aqui estou usando a padrão que é o `tensorflow`. Mas ele aceita `jax`, `pytorch` e `paddle` também.

In [2]:
import matplotlib.pyplot as plt

# Troca o backend para o pytorch. Caso queira usar o PaddlePaddle, basta trocar pytorch por paddle (depois de instalá-lo).
import os
os.environ['DDE_BACKEND'] = 'pytorch'

import deepxde as dde

# Troca o tipo de ponto flutuante para 64 bit, originalmente é 32 bit (essa alteração é necessária)
dde.config.set_default_float('float64')

import numpy as np

# Caso precise usar alguma função específica do pytorch (não é o caso deste notebook)
import torch
# Exemplo: caso precise usar um seno, deve-se usar: torch.sin(...)

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


Set the default float type to float64


---
**1$^\mathrm{a}$ Etapa**

A primeira parte do nosso processo de solução no DeepXDE é definir a equação diferencial a se trabalhar.

O problema de valor inicial é dado pela equação $y'(t)+3t^2 y(t)=6t^2$ e condição inicial $y(0)=0$.

Começamos definindo a função `ode` com duas entradas: (1) a variável independente `t` e (2) a variável dependente `y`.

Para escrevemos a equação diferencial precisamos da derivada da variável dependente. No DeepXDE a 1$^\mathrm{a}$ derivada é calculada utilizando o comando `dde.grad.jacobian(y, t)`.

Por fim retornamos a equação na forma $y'+3t^2 y-6t^2,$ uma vez que o objetivo da rede é minimizar este termo de forma que ele esteja próximo de zero.

In [3]:
def ode(t, y):
    dy_dt = dde.grad.jacobian(y, t)
    return dy_dt + 3 * t**2 * y - 6 * t**2

---
**2$^\mathrm{a}$ Etapa**

Vamos agora definir a geometria do nosso domínio e a condição inicial.

Como estamos lidando com uma EDO, nossa geometria será um intervalo na reta $\mathbb{R}$. Fazemos essa definição utilizando o comando `dde.geometry.TimeDomain(a, b)`.

Em seguida, como estamos lidando com uma condição inicial à esquerda (o usual) vamos definir uma função `boundary_l` que retorna o ponto na fronteira $t=a$.

Finalmente utilizando o módulo de condições iniciais e de fronteira do DeepXDE chamamos a função `dde.icbc.IC(...)` cujas entradas são: (1) a geometria; (2) o valor da condição inicial e (3) qual ponto está a condição inicial.

In [4]:
a = 0
b = 2
y_0 = 0

geom = dde.geometry.TimeDomain(a, b)

# define o limite à esquerda do intervalo
def boundary_l(t, on_initial):
    return on_initial and dde.utils.isclose(t[0], a)

ic1 = dde.icbc.IC(geom, lambda x: y_0, boundary_l)

---
**3$^\mathrm{a}$ Etapa**

Neste momento devemos montar a estrutura da nossa rede neural.

Primeiro passo é utilizar o método `TimePDE` para incluir as informações que definimos no passo anterior: (1) geometria; (2) equação diferencial; (3) condições iniciais e (4) a quantidade de pontos amostrados aleatoriamente no interior do domínio e na fronteira.

Em seguida devemos escolher os parâmetros da rede como: (1) número de camadas e neurônios, (2) função de ativação e (3) o inicializador dos pesos.

Por último compilamos o modelo para fazer a mágica acontecer.

In [ ]:
data = dde.data.TimePDE(geom, ode, ic1, num_domain=64, num_boundary=1)

layer_size = [1] + [50] * 4 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(iterations=10000)

dde.saveplot(losshistory, train_state, issave=False, isplot=False)

Compiling model...


---
**4$^\mathrm{a}$ Etapa**

Finalmente, após fazer os cálculos necessários, vamos verificar se a rede produziu a solução esperada comparando com a solução conhecida.

Aqui vamos utilizar a função `geom.uniform_points` para gerar a malha necessária para os gráficos e `model.predict` para gerar as imagens dos pontos da malha.

In [ ]:
def func(t):
     return 2 - 2 * np.exp(-t**3)

x = geom.uniform_points(30, True)
y = model.predict(x)
y_test = func(x)
plt.figure()
plt.plot(x, y,x,y_test,'o')
plt.xlabel("t")
plt.ylabel("y(t)")
plt.show()

---
**Objetivo dos próximos dois exemplos:** resolver um problema de valor inicial e um problema de valor de contorno envolvendo uma EDO de 2a ordem. Veremos as nuâncias de como tratar cada um dos casos.

---
### EXEMPLO 2

**Objetivo:** Neste exemplo vamos resolver o problema de valor inicial dado pela equação não-homogênea $$y''(t)+5 y'(t) + 6 y(t)=e^{-t}$$ com condições iniciais $y(0)=-1$ e $y'(0)=2$.

* Esse PVI tem solução analítica dada por $y(t)=\frac{1}{2} e^{-3t} \left(1-4e^{t}+e^{2t}\right)$.

---
**1$^\mathrm{a}$ Etapa**

Começamos definindo a equação diferencial a se trabalhar.

Começamos definindo a função `ode` com duas entradas: (1) a variável independente `t` e (2) a variável dependente `y`.

Para escrevemos a equação diferencial precisamos da derivada da variável dependente. No DeepXDE a 1$^\mathrm{a}$ derivada é calculada utilizando o comando `dde.grad.jacobian(y, t)` e a 2$^\mathrm{a}$ derivada é calculada utilizando o comando `dde.grad.hessian(y, t)`.

Por fim retornamos a equação na forma $y''+5 y'+ 6 y=e^{-t},$ uma vez que o objetivo da rede é minimizar este termo de forma que ele esteja próximo de zero.

*Obs.:* como estamos usando o `tensorflow` como *backend* precisamos chamar a exponencial definida pelo `tensorflow`. Caso seja usada a versão do `numpy` o código retornará alguma erro e não irá rodar.

In [ ]:
def ode(t, y):
    dy_dt = dde.grad.jacobian(y, t)
    d2y_dt2 = dde.grad.hessian(y, t)
    return d2y_dt2 + 5 * dy_dt + 6 * y - torch.exp(-t) # exponencial definida no tensorflow

---
**2$^\mathrm{a}$ Etapa**

Vamos agora definir a geometria do nosso domínio e as condições iniciais.

Novamente, como estamos lidando com uma EDO, nossa geometria será um intervalo na reta $\mathbb{R}$. Fazemos essa definição utilizando o comando `dde.geometry.TimeDomain(a, b)`.

Em seguida, como estamos lidando com duas condições iniciais à esquerda (o usual) vamos definir uma função `boundary_l` que retorna o ponto na fronteira $t=a$.

Finalmente utilizando o módulo de condições iniciais e de fronteira do DeepXDE chamamos a função `dde.icbc.IC(...)` cujas entradas são: (1) a geometria; (2) o valor da condição inicial e (3) qual ponto está a condição inicial.

Aqui temos um detalhe importante, pois o módulo `dde.icbc.IC(...)` é capaz de definir apenas uma condição inicial que é $y(a)=y_0$. Portanto, precisamos definir a 2$^\mathrm{a}$ condição inicial $y'(a)=y_1$ utilizando o módulo `dde.icbc.OperatorBC` que permite definir condições de contorno gerais. Neste caso, através da função `bc_func` calculamos a derivada da rede e fazemos a diferença dela com a condição inicial `y_1`. Isto deve ser feito desta forma, pois o módulo `OperatorBC` deve retornar como entrada uma função nula: `func(inputs, outputs, X) = 0`.

*Obs.: Alternativamente é possível utilizar o módulo `dde.icbc.NeumannBC(...)`, faça o teste depois* ;-)

In [ ]:
a = 0
b = 2
y_0 = -1
y_1 = 2

geom = dde.geometry.TimeDomain(a, b)

def boundary_l(t, on_initial):
    return on_initial and dde.utils.isclose(t[0], a)

# alternativamente, podemos utilizar a função operatorbc para definir a
# condição de contorno da derivada
# def bc_func(inputs, outputs, X):
#     return dde.grad.jacobian(outputs, inputs, i=0, j=None) - y_1

# essa linha poderia ser reescrita como ic1 = dde.icbc.IC(geom, lambda x: y_0, boundary_l)
ic1 = dde.icbc.IC(geom, lambda x: y_0, boundary_l) # define y(t_0)=y_0

ic2 = dde.icbc.NeumannBC(geom, lambda x: -y_1, boundary_l) # define y'(t_0)=y_1

# caso se opte por usar a função operatorbc
#ic2 = dde.icbc.OperatorBC(geom, bc_func, boundary_l) # define y'(t_0)=y_1

---
**3$^\mathrm{a}$ Etapa**

Montamos agora a estrutura da nossa rede neural.

Primeiro passo é utilizar o método `TimePDE` para incluir as informações que definimos no passo anterior: (1) geometria; (2) equação diferencial; (3) condições iniciais e (4) a quantidade de pontos amostrados aleatoriamente no interior do domínio e na fronteira.

Em seguida devemos escolher os parâmetros da rede como: (1) número de camadas e neurônios, (2) função de ativação e (3) o inicializador dos pesos.

Por último compilamos o modelo para fazer a mágica acontecer.

In [ ]:
data = dde.data.TimePDE(geom, ode, [ic1, ic2], num_domain=64, num_boundary=1)
layer_size = [1] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(iterations=10000)

dde.saveplot(losshistory, train_state, issave=False, isplot=False)

---
**4$^\mathrm{a}$ Etapa**

Finalmente, após fazer os cálculos necessários, vamos verificar se a rede produziu a solução esperada comparando com a solução conhecida.

Aqui vamos utilizar a função `geom.uniform_points` para gerar a malha necessária para os gráficos e `model.predict` para gerar as imagens dos pontos da malha.

In [ ]:
def func(t):
    return 0.5 * np.exp(-3*t)*(1 - 4 * np.exp(t) + np.exp(2*t))

x = geom.uniform_points(30, True)
y = model.predict(x)
y_test = func(x)
plt.figure()
plt.plot(x, y,x,y_test,'o')
plt.xlabel("t")
plt.ylabel("y")
plt.show()

---
### EXEMPLO 3

**Objetivo:** Neste exemplo vamos resolver o problema de valor inicial dado pela equação não-homogênea $$y''(t)+9 y(t)=\cos{t}$$ com condições de fronteira $y'(\pi/4)=5$ e $y'(\pi/2)=-5/3$.

* Esse PVC tem solução dada por $y(t)=\frac{1}{72} \left(9 \cos(x) - 37 \cos(3 x) + 2 (17 - 60 \sqrt{2}) \sin(3 x)\right)$.

---
**1$^\mathrm{a}$ Etapa**

Novamente começamos definindo a equação diferencial a se trabalhar.

Começamos definindo a função `ode` com duas entradas: (1) a variável independente `t` e (2) a variável dependente `y`.

Para escrevemos a equação diferencial precisamos da derivada da variável dependente. No DeepXDE a 1$^\mathrm{a}$ derivada é calculada utilizando o comando `dde.grad.jacobian(y, t)` e a 2$^\mathrm{a}$ derivada é calculada utilizando o comando `dde.grad.hessian(y, t)`.

Por fim retornamos a equação na forma $y''+9 y-\cos(t),$ uma vez que o objetivo da rede é minimizar este termo de forma que ele esteja próximo de zero.

In [ ]:
def ode(t, y):
    dy_dt = dde.grad.jacobian(y, t)
    d2y_dt2 = dde.grad.hessian(y, t)
    return d2y_dt2 + 9 * y - torch.cos(t) # lembre-se de usar o cosseno do tensorflow ;)

---
**2$^\mathrm{a}$ Etapa**

Vamos agora definir a geometria do nosso domínio e as condições iniciais.

Novamente, como estamos lidando com uma EDO, nossa geometria será um intervalo na reta $\mathbb{R}$. Fazemos essa definição utilizando o comando `dde.geometry.TimeDomain(a, b)`.

Em seguida, como estamos lidando com duas condições de contorno vamos definir duas funções `boundary_l` e `boundary_r` que retornam os pontos nas fronteiras $t=a$ e $t=b$.

Finalmente iremos utilizar o módulo de condições de contorno de Neumann `dde.icbc.NeumannBC(...)` cujas entradas são: (1) a geometria; (2) o valor da condição inicial e (3) qual ponto está a condição de fronteira.

*Obs.: Note que na condição à esquerda a o valor dado está com o sinal trocado, isto deve sempre acontecer porque o vetor normal deste lado está virado para a esquerda*

In [ ]:
a = np.pi/4
b = np.pi/2
y_0 = 5
y_1 = -5/3

geom = dde.geometry.TimeDomain(a, b)

def boundary_l(t, on_boundary):
    return on_boundary and dde.utils.isclose(t[0], a)

def boundary_r(t, on_boundary):
    return on_boundary and dde.utils.isclose(t[0], b)

ic1 = dde.icbc.NeumannBC(geom, lambda x: -y_0, boundary_l)
ic2 = dde.icbc.NeumannBC(geom, lambda x: y_1, boundary_r)

---
**3$^\mathrm{a}$ Etapa**

Montamos agora a estrutura da nossa rede neural.

Primeiro passo é utilizar o método `TimePDE` para incluir as informações que definimos no passo anterior: (1) geometria; (2) equação diferencial; (3) condições iniciais e (4) a quantidade de pontos amostrados aleatoriamente no interior do domínio e na fronteira.

Em seguida devemos escolher os parâmetros da rede como: (1) número de camadas e neurônios, (2) função de ativação e (3) o inicializador dos pesos.

Por último compilamos o modelo para fazer a mágica acontecer.

In [ ]:
data = dde.data.TimePDE(geom, ode, [ic1, ic2], num_domain=64, num_boundary=2)
layer_size = [1] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(iterations=10000)

dde.saveplot(losshistory, train_state, issave=False, isplot=False)

---
**4$^\mathrm{a}$ Etapa**

Finalmente, após fazer os cálculos necessários, vamos verificar se a rede produziu a solução esperada comparando com a solução conhecida.

Aqui vamos utilizar a função `geom.uniform_points` para gerar a malha necessária para os gráficos e `model.predict` para gerar as imagens dos pontos da malha.

In [ ]:
def func(t):
    return (1/72) * (9*np.cos(x) - 37*np.cos(3*x) + 2*(17 - 60*np.sqrt(2))*np.sin(3*x))

x = geom.uniform_points(30, True)
y = model.predict(x)
y_test = func(x)
plt.figure()
plt.plot(x, y,x,y_test,'o')
plt.xlabel("t")
plt.ylabel("y")
plt.show()